## For the first time restoration


1. Create new user **ausstage** and grant the privileges.<br>
   (The new user name have to to be **'ausstage'**, as the original view has defined the user names, otherwise we can't access the view.)

```
CREATE USER 'ausstage' IDENTIFIED BY 'ACD_Engine!2022';
GRANT ALL ON *.* TO 'ausstage';
FLUSH PRIVILEGES;

```

2. Download the DB dump from secret backend api. Then unzip it

```
https://rdf.ausstage.edu.au/joseki/ausstage_complete_inserts.zip
```

3. Restore the DB from dump sql in force mode. (with restore log)

```
mysql -u ausstage -p -f < script.sql
```

## For the following update

### Setting

In [1]:
import sys

codefolder = "C:\ProjectCollections\Programs\Australia_Cultural_Data_Engine\codes"
sys.path.append(codefolder)

from ausstage import DBExtraction as adb_manip

from general import GeneralFunctions as gen_gf
from general import JsonProcessing as gen_jp
from general import MongoDBManipulation as gen_manip

In [2]:
import os
import shutil
import zipfile
from datetime import datetime
from subprocess import run
from time import time
from urllib.request import urlretrieve

from pymysql.constants import CLIENT
from sqlalchemy import create_engine, text
from sqlalchemy_utils.functions import database_exists, drop_database

In [3]:
download_url = "https://rdf.ausstage.edu.au/joseki/ausstage_complete_inserts.zip"
db_dump_start = download_url.rsplit("/", 1)[-1][:-4]
db_dump_folder = "D:/Program_Data/Australia_Cultural_Data_Engine_Data/australian_live_performance_database/ausstage_dbdump"
remove_prev = False

db_user = "ausstage"
user_pw = "ACD_Engine!2022"
mysql_path = "C:/Program Files/MySQL/MySQL Server 8.0/bin"
restore_fpath = f"{db_dump_folder}/restore.log"

### Download latest database dump (Previous databse removal is optional.)

In [6]:
# Remove history version of dump file (or not)
gen_gf.remove_history(db_dump_folder, db_dump_start, remove_prev)

current_ts = gen_gf.time_now()
dump_name = db_dump_start + "_" + current_ts + ".zip"
db_dump_fpath = os.path.join(db_dump_folder, dump_name)

urlretrieve(download_url, db_dump_fpath)

Keep hitorical version.


('D:/Program_Data/Australia_Cultural_Data_Engine_Data/australian_live_performance_database/ausstage_dbdump\\ausstage_complete_inserts_202209201742.zip',
 <http.client.HTTPMessage at 0x116528a86a0>)

### Unzip Database Dump

In [4]:
unzip_name = "opt/ausstage/latest/webapps/joseki//" + db_dump_start + ".sql"
unzip_rename = db_dump_start + "_" + current_ts + ".sql"
unzip_fpath = os.path.join(db_dump_folder, unzip_rename)

gen_gf.UnzipFile(db_dump_fpath, unzip_name, unzip_fpath)

opt/ausstage/latest/webapps/joseki//ausstage_complete_inserts.sql is unzipped as D:/Program_Data/Australia_Cultural_Data_Engine_Data/australian_live_performance_database/ausstage_dbdump\ausstage_complete_inserts_202203211958.sql.


### Drop Database if it exists

In [5]:
adb_manip.DropDB(
    dbUserName=db_user, dbUserPassword=user_pw, databaseForDeletion="ausstage",
)

Database ausstage exists!
Database 'ausstage' is dropped!


### Restore Databse

In [6]:
cmdline = (
    f'mysql --user={db_user} -p{user_pw} -f < "{unzip_fpath}" > "{restore_fpath}" 2>&1'
)
rc = run(cmdline, cwd=mysql_path, shell=True)
if rc.returncode == 0:
    print("The Databse is restored!")
else:
    print(f"Sorry the database is not restored. Please check {restore_fpath}")
    print(rc.stderr)

The Databse is restored!


## Create Table(s) and Views for Transformation

### Create Table(s) (for unification)

* Contributor(Biography) Unification

1. "Is related to" is ambiguous, it may refer to many different relations.<br>
`ausstage_bio[ausstage_bio.contributorid.isin(['236764', '479206'])]`<br>
Answer: The official definition: "Related to another Contributor through extended family" [Data Entry Manual P31](https://www.ausstage.edu.au/opencms/export/system/modules/au.edu.flinders.ausstage.learn/pages/learn/contribute/manuals/AusStage_Data_Entry_Manual_2021.pdf)<br>
Answer: The "is relative to" in organisation relationship could refer to: A is related to B through extended relations. [e.g. Data Entry Manual P34](https://www.ausstage.edu.au/opencms/export/system/modules/au.edu.flinders.ausstage.learn/pages/learn/contribute/manuals/AusStage_Data_Entry_Manual_2021.pdf)

2. For people with different names...?
3. For people with  different names and different gender ...?? ([Kween Kong](https://www.ausstage.edu.au/pages/contributor/544093)
is also known as [Thomas Fonua](https://www.ausstage.edu.au/pages/contributor/50084)
)

In [ ]:
adb_manip.ExecuteSQLScript(
    dbUserName=db_user,
    dbUserPassword=user_pw,
    db_name="ausstage",
    execute_script_fpath=os.path.join(
        codefolder, "ausstage", "CreateTable_EventProductID.sql"
    ),
)

In [ ]:
adb_manip.ExecuteSQLScript(
    dbUserName=db_user,
    dbUserPassword=user_pw,
    db_name="ausstage",
    execute_script_fpath=os.path.join(
        codefolder, "ausstage", "CreateTable_UnfTables.sql"
    ),
)

In [18]:
ori_tables = ["contributor", "organisation", "venue"]
alsokwn_queries = [""] * len(ori_tables)
with open(os.path.join(codefolder, "ausstage", "Extract_AlsoKwnRelations.sql")) as f:
    for query in f.read().strip().split(";"):
        if "CONTRIBUTORID" in query:
            alsokwn_queries[0] = query + ";"
        elif "ORGANISATIONID" in query:
            alsokwn_queries[1] = query + ";"
        elif "VENUEID" in query:
            alsokwn_queries[2] = query + ";"
unf_tables = ["acde_cont_unf_alsokwn", "acde_org_unf_alsokwn", "acde_venue_unf_alsokwn"]

for ori_table, alsokwn_query, unf_table in zip(ori_tables, alsokwn_queries, unf_tables):
    joining_key = f"{ori_table.upper()}ID"
    serial_attr = f"{joining_key.lower()}_unf_alsokwn"
    source_attr = joining_key
    target_attr = "CHILDID"

    adb_manip.ausstage_Create_UnfTable(
        ori_table=ori_table,
        alsokwn_query=alsokwn_query,
        unf_table=unf_table,
        joining_key=joining_key,
        serial_attr=serial_attr,
        source_attr=source_attr,
        target_attr=target_attr,
        db_con=adb_manip.CreateMySQLEngine(),
        chunksize=10000,
    )

contributorid_unf_alsokwn is filled into table acde_cont_unf_alsokwn.
organisationid_unf_alsokwn is filled into table acde_org_unf_alsokwn.
venueid_unf_alsokwn is filled into table acde_venue_unf_alsokwn.


### Create Views

In [20]:
ausstage_codefolder = os.path.join(codefolder, "ausstage")
createview_fpath = os.path.join(ausstage_codefolder, "CreateViews.sql")

exe_files = []
for r, dirs, files in os.walk(ausstage_codefolder):
    for file in files:
        if file.startswith("CreateView_") and "checkpoint" not in file and "archive" not in file:
            exe_files.append(file)

# as the other views use acde_event view, create it first
exe_files = [f for f in exe_files if 'Event' in f] + [f for f in exe_files if 'Event' not in f]
with open(createview_fpath, "w") as f_writer:
    for exe_file in exe_files:
        with open(os.path.join(ausstage_codefolder, exe_file)) as f_reader:
            f_writer.write(f_reader.read())
            f_writer.write("\n")

In [21]:
createview_fpath = os.path.join(ausstage_codefolder, "CreateViews.sql")

adb_manip.ExecuteSQLScript(
    dbUserName=db_user,
    dbUserPassword=user_pw,
    db_name="ausstage",
    execute_script_fpath=createview_fpath,
)

C:\ProjectCollections\Programs\Australia_Cultural_Data_Engine\codes\ausstage\CreateViews.sql is executed!


### Explore the Name in the Views

In [7]:
ausstage_db = adb_manip.CreateMySQLEngine()

from sqlalchemy import inspect

ausstage_inspector = inspect(ausstage_db)

In [ ]:
for v_name in ausstage_inspector.get_view_names():
    if '_full_' in v_name or "network" in v_name:
        print(v_name)
        for col_name in ausstage_inspector.get_columns(v_name):
            print("\t", col_name.get('name'))